In [10]:
import sys

sys.path.append("../")

# load from local .env file
from dotenv import load_dotenv, find_dotenv

env = load_dotenv(".env", override=True)

# standard python
from typing import List, Dict, Tuple
import hyperopt as hp
import os

# external libraries
from tqdm import tqdm
from rich import (
    print,
)  # nice library that provides improved printing output (overrides default print function)

# external files
from src.reranker import ReRanker
from src.database.weaviate_interface_v4 import WeaviateWCS

from src.evaluation.retrieval_evaluation import execute_evaluation
from src.preprocessor.preprocessing import FileIO

api_key = os.environ["WEAVIATE_API_KEY"]
url = os.environ["WEAVIATE_ENDPOINT"]

In [15]:
# read env vars from local .env file
model_path = "../models/bge-base-finetuned-500"
reranker_path = "BAAI/bge-reranker-base"
model_name_or_path = model_path

# instantiate client
client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path)
reranker = ReRanker(model_name=reranker_path)

# hyperparams
sizes = [128, 256, 512]
limits = [40, 50, 60, 80, 100]

#################
##  START CODE ##
#################

golden_datasets = [
    FileIO.load_json(f"../data/golden_datasets/golden_{size}_hard.json")
    for size in sizes
]
collection_names = [f"Huberman_bge_finetuned_500_{size}" for size in sizes]

/opt/conda/envs/vec/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
def f(x):
    print(x[0], sizes[x[1]], limits[x[2]])
    res = execute_evaluation(
        golden_datasets[x[1]],
        collection_names[x[1]],
        retriever=client,
        reranker=reranker,
        chunk_size=sizes[x[1]],
        alpha=x[0],
        retrieve_limit=limits[x[2]],
        dir_outpath="drive/MyDrive/outfiles4/",
    )
    print(res["hybrid_hit_rate"], x[0], sizes[x[1]], limits[x[2]])
    return (-1) * res["hybrid_hit_rate"]


#################
##  END CODE   ##
#################

In [19]:
x0 = hp.fmin(
    fn=f,
    space=[
        hp.hp.uniform("alpha", 0.2, 0.8),
        hp.hp.randint("size", 0, 3),
        hp.hp.randint("limit", 0, 5),
    ],
    max_evals=50,
)

TPE is being used as the default algorithm.


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

0.2838827089221218 128 60

Queries: 100%|##########| 100/100 [03:54<00:00,  2.34s/it]


Total Processing Time: 3.91 minutes

...

 86%|████████▌ | 43/50 [6:20:51<52:40, 451.44s/trial, best loss: -0.88]  

0.6586518526854168 512 40

Queries:  83%|########2 | 83/100 [06:48<01:23,  4.92s/it]


 86%|████████▌ | 43/50 [6:27:39<1:03:06, 540.93s/trial, best loss: -0.88]


KeyboardInterrupt: 

In [12]:
f([0.54, 2, 2])

0.54 512 60

Queries: 100%|██████████| 100/100 [12:05<00:00,  7.25s/it]


Total Processing Time: 12.09 minutes

0.88 0.54 512 60

-0.88